Step 1: Scrape data

Create a bugtitle:patch data set (json). Read patch with bug links obtained from the kenrel git log, and scrape the corresponding bug titles (total of 6525). Kernel git log was obtained by cloning, Linus' kernel tree, and searching the git log (git log --grep='bugzilla')

In [ ]:
import json
# Read patches to find commit id and bug links
ifile = open("patches_with_bugzilla", "r", encoding = "ISO-8859-1")


tokens = []
buglinks = {} # Map of commitid:buglink
for line in ifile.readlines():
    tokens = line.split(' ')
    if tokens[0] == "commit":
        commitid = tokens[1].rstrip("\n")
    for i, ele in enumerate(tokens):
        pos = ele.find("bugzilla")
        if pos != -1 : 
            buglinks[commitid] = ele

#Read patch description into a map of commitid:patchdesc
tokens = []
patches = {}
ifile.seek(0)

for line in ifile.readlines():
    tokens = line.split(' ')
    if tokens[0] == "commit":
        commitid = tokens[1].rstrip("\n")
        patches[commitid] = ""
    
    for i, ele in enumerate(tokens): # Lines to be ignored
        if (ele.find("commit") != -1) : line = ""
        if (ele.find("Author:") != -1): line = ""
        if (ele.find("Date:") != -1): line = ""
        if (ele.find("Link:") != -1): line = ""
        if (ele.find("Fixes:") != -1): line = ""
        if (ele.find("Reported-by:") != -1): line = ""
        if (ele.find("Signed-off-by:") != -1): line = ""
        if (ele.find("Cc:") != -1): line = ""
        if (ele.find("CC:") != -1): line = ""
        #if (ele.find("URL:") != -1): line = ""
        #if (ele.find("Buglink") != -1): line = ""
        if (ele.find("Acked-by:") != -1): line = ""
        #if (ele.find("Bugzilla:") != 1): line = ""
        if (ele.find("Reviewed-by:") != -1): line = "" 
        if (ele.find("Tested-by:") != -1): line = ""
        if (ele.find("Suggested-by:") != -1): line = ""

    patches[commitid] = patches[commitid] + line     
    
print(len(patches))

ifile.close()

In [ ]:
import requests
from urllib.parse import urlparse
from bs4 import BeautifulSoup


requests.packages.urllib3.disable_warnings()
bugtitle = {} # Map of commitid: bugtitle
bugpatch = {}
count = 0
for id, bugurl in buglinks.items():
    if count >= 1000:
        break
    if id in bugpatch: # Already processed
        continue 
    if not urlparse(bugurl).netloc: #invalid url
         continue
    try:
            response = requests.get(bugurl, verify=False, timeout=10)
    except:
        continue
    
    # Extracting the source code of the page.
    data = response.text

    # Passing the source code to BeautifulSoup to create a BeautifulSoup object for it.
    soup = BeautifulSoup(data, 'lxml')
    title = soup.find('title')
    if title is None:
        bugtitle[id] = ""
    else:
        bugtitle[id] = title.string

    count = count + 1

# Create a new map of bugtitle:patchdesc

for i,t in bugtitle.items():
    if i in patches:
        bugpatch[t] = patches[i]

        
# Write out to json file
with open('bugpatch.json', 'w+') as fp:
    json.dump(bugpatch, fp)
    

# Check if json dump was okay
#with open("bugpatch.json", "r") as fp:
#    data = json.load(fp)
    
#for k, v in data.items():
#    print(k, v)

